In [1]:
import os
import json
import itertools

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
os.chdir("..")
os.chdir("..")

# Parameters

In [10]:
antibiotic_dictionary_file = open("data/antibiotics.json", "r")
antibiotic_dictionary = json.load(antibiotic_dictionary_file)
antibiotics = antibiotic_dictionary["Staphylococcus aureus"]

min_instances = 10

In [11]:
file = "data/unprocessed/raw/s_aureus_driams.csv"

bac = pd.read_csv(file)
bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,Clindamycin,Ceftriaxone,Cefepime,Piperacillin-Tazobactam,Meropenem,Imipenem,Tobramycin,Fusidic acid,Oxacillin,Vancomycin
0,283.000000,266.0,208.333333,195.000000,175.0,250.666667,278.5,311.333333,262.500000,242.5,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1,154.000000,166.5,155.000000,196.000000,142.0,144.333333,180.5,211.000000,173.500000,192.5,...,0.0,1.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,0.0
2,165.000000,132.5,213.000000,178.000000,164.5,173.333333,155.0,185.333333,142.500000,164.5,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
3,728.666667,688.5,671.333333,640.000000,724.0,714.333333,602.5,605.333333,667.000000,566.0,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
4,946.000000,864.0,887.666667,1049.500000,1003.0,936.000000,813.0,799.666667,780.500000,767.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760,723.000000,688.0,724.000000,784.000000,603.5,688.000000,638.5,694.500000,747.000000,610.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
3761,324.000000,327.0,384.500000,237.000000,301.0,239.333333,224.5,253.000000,227.000000,236.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,0.0
3762,607.000000,584.0,566.000000,679.666667,623.5,599.333333,537.0,586.500000,566.666667,591.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
3763,1667.000000,1587.0,1742.333333,1872.000000,1971.0,1791.000000,1580.5,1434.333333,1428.500000,1516.0,...,0.0,1.0,1.0,1.0,1.0,1.0,NaN,0.0,1.0,0.0


# Preprocessing

In [12]:
malditof = bac[bac.columns.drop(list(bac.filter(regex='[^0-9]')))]
malditof

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,283.000000,266.0,208.333333,195.000000,175.0,250.666667,278.5,311.333333,262.500000,242.5,...,803.0,846.0,786.0,788.0,752.0,855.0,802.0,762.0,830.0,322.787651
1,154.000000,166.5,155.000000,196.000000,142.0,144.333333,180.5,211.000000,173.500000,192.5,...,190.0,215.0,117.0,141.0,148.0,150.0,182.0,159.0,153.0,44.302826
2,165.000000,132.5,213.000000,178.000000,164.5,173.333333,155.0,185.333333,142.500000,164.5,...,68.0,62.0,61.0,71.0,82.0,76.0,75.0,82.0,55.0,14.067113
3,728.666667,688.5,671.333333,640.000000,724.0,714.333333,602.5,605.333333,667.000000,566.0,...,134.0,174.0,153.0,111.0,139.0,121.0,115.0,136.0,127.0,30.151767
4,946.000000,864.0,887.666667,1049.500000,1003.0,936.000000,813.0,799.666667,780.500000,767.0,...,224.0,214.0,173.0,195.0,221.0,223.0,197.0,184.0,166.0,56.105057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760,723.000000,688.0,724.000000,784.000000,603.5,688.000000,638.5,694.500000,747.000000,610.0,...,897.0,901.0,941.0,912.0,951.0,1044.0,1074.0,1075.0,1131.0,314.202138
3761,324.000000,327.0,384.500000,237.000000,301.0,239.333333,224.5,253.000000,227.000000,236.0,...,353.0,291.0,368.0,364.0,339.0,314.0,311.0,329.0,321.0,116.624536
3762,607.000000,584.0,566.000000,679.666667,623.5,599.333333,537.0,586.500000,566.666667,591.0,...,463.5,424.0,372.0,379.0,449.0,426.0,467.0,453.0,540.0,133.716472
3763,1667.000000,1587.0,1742.333333,1872.000000,1971.0,1791.000000,1580.5,1434.333333,1428.500000,1516.0,...,305.0,257.0,261.0,354.0,277.0,318.0,333.0,334.0,365.0,90.348109


In [13]:
trimmed_bac = pd.DataFrame(np.column_stack([malditof, bac[antibiotics]]))
trimmed_bac.columns = list(malditof.columns) + antibiotics
trimmed_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9994,9995,9996,9997,9998,9999,Oxacillin,Ceftriaxone,Clindamycin,Fusidic acid
0,283.000000,266.0,208.333333,195.000000,175.0,250.666667,278.5,311.333333,262.500000,242.5,...,752.0,855.0,802.0,762.0,830.0,322.787651,0.0,0.0,0.0,0.0
1,154.000000,166.5,155.000000,196.000000,142.0,144.333333,180.5,211.000000,173.500000,192.5,...,148.0,150.0,182.0,159.0,153.0,44.302826,1.0,1.0,0.0,0.0
2,165.000000,132.5,213.000000,178.000000,164.5,173.333333,155.0,185.333333,142.500000,164.5,...,82.0,76.0,75.0,82.0,55.0,14.067113,0.0,0.0,0.0,0.0
3,728.666667,688.5,671.333333,640.000000,724.0,714.333333,602.5,605.333333,667.000000,566.0,...,139.0,121.0,115.0,136.0,127.0,30.151767,0.0,0.0,1.0,0.0
4,946.000000,864.0,887.666667,1049.500000,1003.0,936.000000,813.0,799.666667,780.500000,767.0,...,221.0,223.0,197.0,184.0,166.0,56.105057,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760,723.000000,688.0,724.000000,784.000000,603.5,688.000000,638.5,694.500000,747.000000,610.0,...,951.0,1044.0,1074.0,1075.0,1131.0,314.202138,0.0,0.0,0.0,0.0
3761,324.000000,327.0,384.500000,237.000000,301.0,239.333333,224.5,253.000000,227.000000,236.0,...,339.0,314.0,311.0,329.0,321.0,116.624536,1.0,1.0,1.0,0.0
3762,607.000000,584.0,566.000000,679.666667,623.5,599.333333,537.0,586.500000,566.666667,591.0,...,449.0,426.0,467.0,453.0,540.0,133.716472,0.0,0.0,0.0,0.0
3763,1667.000000,1587.0,1742.333333,1872.000000,1971.0,1791.000000,1580.5,1434.333333,1428.500000,1516.0,...,277.0,318.0,333.0,334.0,365.0,90.348109,1.0,1.0,0.0,0.0


In [14]:
print(trimmed_bac.isna().sum())
print("\nTotal number of NA data: " + str(trimmed_bac.isna().sum().sum()))

2000              0
2001              0
2002              0
2003              0
2004              0
               ... 
9999              0
Oxacillin         1
Ceftriaxone     150
Clindamycin     215
Fusidic acid     25
Length: 8004, dtype: int64

Total number of NA data: 391


In [15]:
scrubbed_bac = trimmed_bac.dropna()
scrubbed_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9994,9995,9996,9997,9998,9999,Oxacillin,Ceftriaxone,Clindamycin,Fusidic acid
0,283.000000,266.0,208.333333,195.000000,175.0,250.666667,278.5,311.333333,262.500000,242.5,...,752.0,855.0,802.0,762.0,830.0,322.787651,0.0,0.0,0.0,0.0
1,154.000000,166.5,155.000000,196.000000,142.0,144.333333,180.5,211.000000,173.500000,192.5,...,148.0,150.0,182.0,159.0,153.0,44.302826,1.0,1.0,0.0,0.0
2,165.000000,132.5,213.000000,178.000000,164.5,173.333333,155.0,185.333333,142.500000,164.5,...,82.0,76.0,75.0,82.0,55.0,14.067113,0.0,0.0,0.0,0.0
3,728.666667,688.5,671.333333,640.000000,724.0,714.333333,602.5,605.333333,667.000000,566.0,...,139.0,121.0,115.0,136.0,127.0,30.151767,0.0,0.0,1.0,0.0
4,946.000000,864.0,887.666667,1049.500000,1003.0,936.000000,813.0,799.666667,780.500000,767.0,...,221.0,223.0,197.0,184.0,166.0,56.105057,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760,723.000000,688.0,724.000000,784.000000,603.5,688.000000,638.5,694.500000,747.000000,610.0,...,951.0,1044.0,1074.0,1075.0,1131.0,314.202138,0.0,0.0,0.0,0.0
3761,324.000000,327.0,384.500000,237.000000,301.0,239.333333,224.5,253.000000,227.000000,236.0,...,339.0,314.0,311.0,329.0,321.0,116.624536,1.0,1.0,1.0,0.0
3762,607.000000,584.0,566.000000,679.666667,623.5,599.333333,537.0,586.500000,566.666667,591.0,...,449.0,426.0,467.0,453.0,540.0,133.716472,0.0,0.0,0.0,0.0
3763,1667.000000,1587.0,1742.333333,1872.000000,1971.0,1791.000000,1580.5,1434.333333,1428.500000,1516.0,...,277.0,318.0,333.0,334.0,365.0,90.348109,1.0,1.0,0.0,0.0


In [16]:
print(scrubbed_bac.isna().sum())
print("\nTotal number of NA data: " + str(scrubbed_bac.isna().sum().sum()))

2000            0
2001            0
2002            0
2003            0
2004            0
               ..
9999            0
Oxacillin       0
Ceftriaxone     0
Clindamycin     0
Fusidic acid    0
Length: 8004, dtype: int64

Total number of NA data: 0


In [17]:
value_counts = scrubbed_bac[antibiotics].value_counts()
df_value_counts = value_counts.rename("Count").to_frame().reset_index()
to_drop = df_value_counts.loc[df_value_counts["Count"] < min_instances].drop("Count", axis=1)
to_drop

,Oxacillin,Ceftriaxone,Clindamycin,Fusidic acid
8,1.0,0.0,0.0,0.0


In [18]:
reduced_aux_bac = pd.merge(scrubbed_bac, to_drop, indicator=True, how='outer', on=antibiotics)
reduced_bac = reduced_aux_bac.loc[reduced_aux_bac["_merge"] == "left_only"].drop("_merge", axis=1)
reduced_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9994,9995,9996,9997,9998,9999,Oxacillin,Ceftriaxone,Clindamycin,Fusidic acid
0,283.000000,266.000000,208.333333,195.000000,175.000000,250.666667,278.500000,311.333333,262.500000,242.500000,...,752.0,855.0,802.0,762.0,830.0,322.787651,0.0,0.0,0.0,0.0
1,165.000000,132.500000,213.000000,178.000000,164.500000,173.333333,155.000000,185.333333,142.500000,164.500000,...,82.0,76.0,75.0,82.0,55.0,14.067113,0.0,0.0,0.0,0.0
2,946.000000,864.000000,887.666667,1049.500000,1003.000000,936.000000,813.000000,799.666667,780.500000,767.000000,...,221.0,223.0,197.0,184.0,166.0,56.105057,0.0,0.0,0.0,0.0
3,760.000000,770.666667,690.000000,725.000000,745.666667,682.500000,728.500000,607.333333,632.000000,604.000000,...,292.0,273.0,279.0,331.0,271.0,86.489238,0.0,0.0,0.0,0.0
4,148.333333,65.000000,107.333333,103.500000,99.500000,95.000000,126.500000,96.000000,80.333333,150.000000,...,304.0,257.0,341.0,424.0,333.0,114.398290,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3522,907.500000,870.666667,993.000000,1037.500000,966.333333,934.000000,890.666667,782.500000,778.500000,799.000000,...,337.0,316.0,294.0,273.0,279.0,80.907052,0.0,0.0,1.0,1.0
3523,1613.666667,1316.500000,1385.333333,1460.000000,1492.000000,1843.333333,1899.000000,1687.333333,1520.500000,1486.000000,...,400.0,460.0,363.0,386.0,380.0,94.600112,0.0,0.0,1.0,1.0
3524,1025.000000,1043.500000,1072.333333,1102.000000,1267.500000,1336.666667,1178.000000,1082.000000,1061.500000,1042.500000,...,569.0,622.0,629.0,602.0,525.0,189.229164,0.0,0.0,1.0,1.0
3525,743.000000,762.000000,711.666667,735.000000,790.666667,684.000000,695.500000,743.333333,892.500000,795.333333,...,29.0,20.0,21.0,25.0,16.0,2.375707,0.0,0.0,1.0,1.0


In [19]:
x = reduced_bac[reduced_bac.columns.drop(list(reduced_bac.filter(regex='[^0-9]')))]
y = reduced_bac[antibiotics]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y[antibiotics])

In [20]:
minmax_scaler = MinMaxScaler()
normalized_x_train = minmax_scaler.fit_transform(x_train)
normalized_x_test = minmax_scaler.transform(x_test)

In [14]:
train = pd.DataFrame(np.column_stack([normalized_x_train, y_train]))
train.columns = list(malditof.columns) + antibiotics

test = pd.DataFrame(np.column_stack([normalized_x_test, y_test]))
test.columns = list(malditof.columns) + antibiotics

# Saving Files

In [15]:
# file_name_ext = os.path.basename(file)
# file_name = os.path.splitext(file_name_ext)[0]
# ext = os.path.splitext(file_name_ext)[1]
# train_name = "data/processed/raw/train_"+file_name+ext
# test_name = "data/processed/raw/test_"+file_name+ext

# train.to_csv(train_name, index=False, mode='w')
# test.to_csv(test_name, index=False, mode='w')